In [1]:
import emoji
import pandas as pd
import numpy as np

In [2]:
emoji_dictionary={
    "0":"\u2764\uFE0F",
    "1":":baseball:",
    "2":":grinning_face_with_big_eyes:",
    "3":":disappointed_face:",
    "4":":fork_and_knife:",
    "5":":hundred_points:"
    
}

In [3]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯


In [4]:
train=pd.read_csv('train_emoji.csv',header=None)
test=pd.read_csv('test_emoji.csv',header=None)

In [5]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [6]:
#lets print sentences with emojis
data=train.values
print(data.shape)

(132, 4)


In [7]:
X_train=train[0]
Y_train=train[1]

X_test=test[0]
Y_test=test[1]

In [8]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


`<b>Step-3: Converting Sentences into Embeddings</b>

In [9]:
f=open('glove.6B.50d.txt',encoding='utf-8')

In [10]:
embeddings_index = {}
cnt=0
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float')
    embeddings_index[word]=coefs #made dictionary for every 6 billion word

f.close()

In [11]:
emb_dim=embeddings_index["eat"].shape[0]

<b> Step-4 Converting Sentences into vectors.(Embedding layer output) </b>

In [12]:
def embedding_output(X):
    maxLen=10
    embedding_out=np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split() #splitting ixth sentence
        for ij in range(len(X[ix])):
            #go to every word in the current (ix) sentence
            
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
                
    return embedding_out

In [13]:
embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)

C:\Users\verma\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#these both matrix will be three dimensional

In [15]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [16]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)

<b> Step-5: RNN/LSTM Architecuture </b>

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import * 

In [18]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True)) #return sequence for giving to the lstm which is going to be stacked#10 word sentence with 50 dimensional-each word
model.add(Dropout(0.4))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [23]:
#trainmodel
import os
#path="training_1/cp.ckpt"
#checkpoint_dir = os.path.dirname(path)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint=ModelCheckpoint("model.hdf5",monitor='val_loss',verbose=True,save_best_only=True,mode='min')
earlystop=EarlyStopping(monitor='val_acc',patience=10)
hist=model.fit(embeddings_matrix_train,Y_train,epochs=40,batch_size=32,shuffle=True,validation_split=0.1,callbacks=[checkpoint,earlystop])

Epoch 1/40
3/4 [=====================>........] - ETA: 0s - loss: 1.6177 - acc: 0.1562
Epoch 00001: val_loss improved from inf to 1.62885, saving model to model.hdf5
4/4 [==============================] - 2s 461ms/step - loss: 1.6141 - acc: 0.1780 - val_loss: 1.6288 - val_acc: 0.1429
Epoch 2/40
4/4 [==============================] - ETA: 0s - loss: 1.5281 - acc: 0.3305
Epoch 00002: val_loss did not improve from 1.62885
4/4 [==============================] - 0s 31ms/step - loss: 1.5281 - acc: 0.3305 - val_loss: 1.6498 - val_acc: 0.2143
Epoch 3/40
1/4 [======>.......................] - ETA: 0s - loss: 1.4886 - acc: 0.3750
Epoch 00003: val_loss did not improve from 1.62885
4/4 [==============================] - 0s 25ms/step - loss: 1.5272 - acc: 0.3136 - val_loss: 1.6773 - val_acc: 0.2143
Epoch 4/40
1/4 [======>.......................] - ETA: 0s - loss: 1.4724 - acc: 0.3750
Epoch 00004: val_loss did not improve from 1.62885
4/4 [==============================] - 0s 23ms/step - loss: 1.503

In [24]:
pred=model.predict_classes(embeddings_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [25]:
print(pred)

[4 3 3 0 2 2 3 2 4 2 1 2 0 3 1 3 3 2 3 2 0 0 4 0 3 3 2 0 1 2 0 1 0 2 0 1 2
 3 4 2 2 0 0 1 2 0 3 2 3 1 3 3 3 2 3 0]


In [26]:
#model.load_weights(path)

In [27]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.2167 - acc: 0.6071


[1.2166780233383179, 0.6071428656578064]

In [28]:
for i in range(30):
    print(' '.join(X_train[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_train[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

never talk to me again
😞
🍴
I am proud of your achievements
😃
😞
It is the worst day in my life
😞
😞
Miss you so much
❤️
❤️
food is life
🍴
😃
I love you mum
❤️
😃
Stop saying bullshit
😞
😞
congratulations on your acceptance
😃
😃
The assignment is too long
😞
🍴
I want to go play
⚾
😃
she did not answer my text
😞
⚾
Your stupidity has no limit
😞
😃
how many points did he score
⚾
❤️
my algorithm performs poorly
😞
😞
I got approved
😃
⚾
Stop shouting at me
😞
😞
Sounds like a fun plan ha ha
😃
😞
no one likes him
😞
😃
the game just finished
⚾
😞
I will celebrate soon
😃
😃
So sad you are not coming
😞
❤️
She is my dearest love
❤️
❤️
Good job
😃
🍴
It was funny lol
😃
❤️
candy is life
😃
😞
The chicago cubs won again
⚾
😞
I am hungry
🍴
😃
I am so excited to see you after so long
😃
❤️
you did well on you exam
😃
⚾
lets brunch some day
🍴
😃


In [29]:
with open("model.json","w") as file:
    file.write(model.to_json())

<b>Django Integration</b>

In [70]:
from tensorflow.keras.models import model_from_json

In [72]:
with open("model.json","r") as file:
    model=model_from_json(file.read())

In [150]:
test_str="how are you doing?"
X=pd.Series([test_str])

In [151]:
emb_X=embedding_output(X)

In [152]:
p=model.predict_classes(emb_X)

In [153]:
p[0]

1

In [154]:
print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

how are you doing?
⚾
